In [21]:
import os
import pandas as pd

train_data_path = os.getenv('HOME')+"/aiffel/dktc/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [22]:
train_data.nunique()

idx             3950
class              4
conversation    3846
dtype: int64

In [23]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화'], dtype=object)

In [24]:
len(train_data['conversation'].values[0])

242

In [25]:
start = 0 
for i in train_data['conversation'].values:
    length = max(start,len(i))
    start = len(i)
print(length)

225


In [26]:

df = train_data
df.loc[ df['class']=='협박 대화', 'idx'] = 0
df.loc[ df['class']=='기타 괴롭힘 대화', 'idx'] = 3
df.loc[ df['class']=='갈취 대화', 'idx'] = 1
df.loc[ df['class']=='직장 내 괴롭힘 대화', 'idx'] = 2

df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,1,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,2,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,1,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


### DistilBERT
-  BERT를 직접 사용하지 않고 BERT의 동작을 모방하는 단순화된 모델(BERT 매개변수의 절반만 가짐)인 DistilBERT를 사용하여 작업
- 매개변수가 절반만 있음에도 불구하고 DistilBERT의 성능은 BERT의 성능과 매우 비슷합니다
-  BERT는 텍스트를 벡터로 인코딩하는 데 사용
- BERT를 기반으로 구축한 분류 모델은 ML 알고리즘을 사용하여 이러한 벡터를 분류하는 것으로 구성
- 전처리 단계로 텍스트에 BERT를 적용한 다음 이러한 벡터를 분류하는 ML 모델을 구축합니다.
- BERT로 시작하는 모델을 구축하고(모델을 훈련할 때 매개변수를 고정함) BERT 위에 이러한 벡터를 분류하는 레이어를 구축합니다.


- LLM(대형 언어 모델)의 경우 데이터를 수동으로 더 이상 처리할 필요가 없습니다. 토크나이저, 즉 텍스트를 "숫자"로 인코딩하는 레이어가 모델과 함께 제공되기 때문입니다.


- 모델을 만들어 보겠습니다. 이를 위해서는 먼저 변환기 라이브러리에서 BERT 레이어를 가져와야 합니다. 훈련 중에 매개변수가 훈련되지 않도록 구성합니다.

In [80]:
import pandas as pd
import numpy as np

X= df['conversation']
y = df['idx']

# Show the data
X[0]

'지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.\n 정말 잘못했습니다.\n 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.\n 죄송합니다. 정말 잘못했습니다.\n 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.\n 선택 못하겠습니다. 한번만 도와주세요.\n 그냥 다 죽여버려야겠군. 이의 없지?\n 제발 도와주세요.'

In [81]:
# distilBERT tokenizer
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [82]:
import tensorflow as tf
# DistilBertTokenizer로 모든 문장 토큰화
X_tf = [tokenizer(text, padding='max_length', max_length = 225, truncation=True)['input_ids'] for text in X]
X_tf = np.array(X_tf, dtype='int32')
# Train/test split
from sklearn.model_selection import train_test_split
X_tf_train, X_tf_test, y_tf_train, y_tf_test = train_test_split(X_tf, y, test_size=0.3, random_state=42, stratify=y)
print('Shape of training data: ',X_tf_train.shape)
print('Shape of test data: ',X_tf_test.shape)

Shape of training data:  (2765, 225)
Shape of test data:  (1185, 225)


In [ ]:
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [113]:

# Get BERT layer
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
config = transformers.DistilBertConfig(dropout=0.2, attention_dropout=0.2)
dbert_tf = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config, trainable=False)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [114]:
# Let's create a sample of size 5 from the training data
sample = X_tf_train[0:5]
print('Object type: ', type(dbert_tf(sample)))
print('Output format (shape): ',dbert_tf(sample)[0].shape)
print('Output used as input for the classifier (shape): ', dbert_tf(sample)[0][:,0,:].shape)

Object type:  <class 'transformers.modeling_tf_outputs.TFBaseModelOutput'>
Output format (shape):  (5, 225, 768)
Output used as input for the classifier (shape):  (5, 768)


In [118]:
from tensorflow.keras import models, layers, metrics

input_ids_in = layers.Input(shape=(225,), name='input_token', dtype='int32')

x = dbert_tf(input_ids=input_ids_in)[0][:,0,:]
x = layers.Dropout(0.5, name='dropout')(x)
x = layers.Dense(64, activation='relu', name='dense2')(x)
x = layers.Dense(4, activation='softmax', name='classification')(x)

model_tf = models.Model(inputs=input_ids_in, outputs = x, name='ClassificationModelTF')

model_tf.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=[metrics.SparseCategoricalAccuracy()])
model_tf.summary()

Model: "ClassificationModelTF"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_token (InputLayer)     [(None, 225)]             0         
_________________________________________________________________
tf_distil_bert_model_4 (TFDi TFBaseModelOutput(last_hi 66362880  
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense2 (Dense)               (None, 64)                49216     
_________________________________________________________________
classification (Dense)       (None, 4)                 260       
Total params: 66,412,356
Trainable params: 49,476
Non-trainable params: 66,362,880
____________________________

In [119]:
from datetime import datetime
# Train the model
start_time = datetime.now()
history = model_tf.fit(X_tf_train, y_tf_train, batch_size=64, shuffle=True, epochs=100, validation_data=(X_tf_test, y_tf_test))
end_time = datetime.now()

Epoch 1/100
44/44 [==============================] - 37s 725ms/step - loss: 1.4194 - sparse_categorical_accuracy: 0.2720 - val_loss: 1.3829 - val_sparse_categorical_accuracy: 0.3105
Epoch 2/100
44/44 [==============================] - 30s 696ms/step - loss: 1.3946 - sparse_categorical_accuracy: 0.2749 - val_loss: 1.3710 - val_sparse_categorical_accuracy: 0.3072
Epoch 3/100
44/44 [==============================] - 30s 682ms/step - loss: 1.3867 - sparse_categorical_accuracy: 0.2626 - val_loss: 1.3739 - val_sparse_categorical_accuracy: 0.2920
Epoch 4/100
44/44 [==============================] - 30s 691ms/step - loss: 1.3797 - sparse_categorical_accuracy: 0.2774 - val_loss: 1.3713 - val_sparse_categorical_accuracy: 0.2827
Epoch 5/100
44/44 [==============================] - 30s 691ms/step - loss: 1.3739 - sparse_categorical_accuracy: 0.2922 - val_loss: 1.3681 - val_sparse_categorical_accuracy: 0.3055
Epoch 6/100
44/44 [==============================] - 30s 688ms/step - loss: 1.3710 - spars

KeyboardInterrupt: 

In [ ]:
. fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5)) 
ax[0].set(title='Loss') 
ax[0].plot( History.history['loss'], label='Training') 
ax[0].plot(history.history['val_loss'], label='Validation') 
ax[0].legend(loc="오른쪽 위" ) 

ax[1].set(title='정확도') 
ax[1].plot(history.history['sparse_categorical_accuracy'], label='Training') 
ax[1].plot(history.history['val_sparse_categorical_accuracy' ], label='검증') 
ax[1].legend(loc="오른쪽 아래")

In [ ]:
Accuracy_tf = History.history['val_sparse_categorical_accuracy'][-1] 
print('정확도 훈련 데이터: {:.1%}'.format(history.history['sparse_categorical_accuracy'][-1])) 
print('정확도 테스트 데이터: {:.1%}'.format(history.history['val_sparse_categorical_accuracy'][-1])) 
print('훈련 시간: {:.1f}s(또는 {:.1f}분)'.format (training_time_tf, training_time_tf/60))

In [ ]:
model_tf.save('model_tf.h5', save_format='h5')
model_tf2 = models.load_model('model_tf.h5', custom_objects={'TFDistilBertModel': dbert_tf})
model_tf2.summary()

In [ ]:
## V. Sanh et al (2019), “DistilBERT, a Distilled Version of BERT: Smaller, Faster, Cheaper and Lighter” (https://arxiv.org/abs/1910.01108)

---

---
## Hugging face Reference

In [74]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [75]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [76]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [80]:
new_df= df[['conversation','idx']].rename(columns={'idx':'labels','conversation':'text'})

In [81]:
new_df.head()

,text,labels
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1


In [79]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# label을 원핫 인코딩
mlb = MultiLabelBinarizer()
new_df['labels'] = mlb.fit_transform(new_df['labels'].apply(lambda x: [x])).tolist()

# 'label' 컬럼 제거
#new_df.drop('labels', axis=1, inplace=True)

new_df

,text,labels
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,"[1, 0, 0, 0]"
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,"[1, 0, 0, 0]"
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,"[0, 0, 0, 1]"
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,"[0, 1, 0, 0]"
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,"[0, 1, 0, 0]"
...,...,...
3945,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,"[0, 0, 0, 1]"
3946,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,"[0, 1, 0, 0]"
3947,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,"[0, 0, 1, 0]"
3948,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,"[0, 1, 0, 0]"


In [82]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [92]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        #token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            #'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.data.labels[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [93]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = MultiLabelDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (3950, 3)
TRAIN Dataset: (3160, 2)
TEST Dataset: (790, 2)


In [94]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [95]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
#768
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 128)
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = torch.nn.Linear(128, 4)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistillBERTClass()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [96]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [97]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [90]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    
    for _,data in enumerate(training_loader, 0):
        ids = data['ids']
        mask = data['mask']
        targets = data['targets']
        #token_type_ids = data['token_type_ids']

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 100 steps: {loss_step}")
            print(f"Training Accuracy per 100 steps: {accu_step}%")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [91]:
for epoch in range(EPOCHS):
    train(epoch)

Training Loss per 100 steps: 1.4122552871704102
Training Accuracy per 100 steps: 0.0%
Training Loss per 100 steps: 1.397143816003705
Training Accuracy per 100 steps: 23.26732673267327%
Training Loss per 100 steps: 1.398432371035144
Training Accuracy per 100 steps: 23.134328358208954%
Training Loss per 100 steps: 1.3998022891358284
Training Accuracy per 100 steps: 22.591362126245848%
Training Loss per 100 steps: 1.3972342531579987
Training Accuracy per 100 steps: 23.19201995012469%
Training Loss per 100 steps: 1.3941161646814404
Training Accuracy per 100 steps: 23.75249500998004%


KeyboardInterrupt: 

In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

---

## ALBERT

In [38]:
df_al = df.rename(columns={'idx':'label','conversation':'text'}).drop(['class'],axis = 1)
df_al

,label,text
0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
3945,3,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,1,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,2,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,1,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [50]:
# 필요한 라이브러리 추가
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AdamW


# 훈련 데이터셋과 테스트 데이터셋으로 분할
train_df, test_df = train_test_split(df_al, test_size=0.2, random_state=42)

# Albert 토크나이저 로드
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['label']
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Albert 모델 로드
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=df_al['label'].nunique())

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [52]:
# 학습 루프
for epoch in range(50):  # 예시로 3 에폭만 진행
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_idx, batch in enumerate(train_dataloader):
        inputs = {
            "input_ids": batch["input_ids"],
            "attention_mask": batch["attention_mask"],
            "labels": batch["label"],
        }
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()

        # 정확도 계산
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == batch["label"]).sum().item()
        total_samples += len(batch["label"])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 한 batch가 끝날 때마다 출력
        if (batch_idx + 1) % 10 == 0:  # 예시로 10 배치마다 출력
            average_loss = total_loss / (batch_idx + 1)
            accuracy = correct_predictions / total_samples
            print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {average_loss}, Accuracy: {accuracy}")

    # 에폭이 끝날 때마다 출력
    average_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}, Training Loss: {average_loss}, Accuracy: {accuracy}")

    # 평가 루프
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_dataloader:
            inputs = {
                "input_ids": batch["input_ids"],
                "attention_mask": batch["attention_mask"],
                "labels": batch["label"],
            }
            outputs = model(**inputs)
            predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(batch["label"].cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Epoch {epoch + 1}, Accuracy: {accuracy}")

Epoch 1, Batch 10, Loss: 1.4341071724891663, Accuracy: 0.1875
Epoch 1, Batch 20, Loss: 1.419752335548401, Accuracy: 0.225
Epoch 1, Batch 30, Loss: 1.399054769674937, Accuracy: 0.2708333333333333
Epoch 1, Batch 40, Loss: 1.4040329337120057, Accuracy: 0.26875
Epoch 1, Batch 50, Loss: 1.4006766295433044, Accuracy: 0.2675
Epoch 1, Batch 60, Loss: 1.4012683471043905, Accuracy: 0.2625
Epoch 1, Batch 70, Loss: 1.4047328574316842, Accuracy: 0.2517857142857143
Epoch 1, Batch 80, Loss: 1.4038457974791527, Accuracy: 0.25625
Epoch 1, Batch 90, Loss: 1.4029256754451327, Accuracy: 0.2625
Epoch 1, Batch 100, Loss: 1.4028579449653626, Accuracy: 0.2625
Epoch 1, Batch 110, Loss: 1.4013782772150907, Accuracy: 0.26477272727272727
Epoch 1, Batch 120, Loss: 1.4033663292725882, Accuracy: 0.26145833333333335
Epoch 1, Batch 130, Loss: 1.40383108487496, Accuracy: 0.25961538461538464
Epoch 1, Batch 140, Loss: 1.4064994479928699, Accuracy: 0.2544642857142857
Epoch 1, Batch 150, Loss: 1.4059185194969177, Accuracy:

Epoch 3, Batch 370, Loss: 1.4069146062876727, Accuracy: 0.25304054054054054
Epoch 3, Batch 380, Loss: 1.4061209490424709, Accuracy: 0.2536184210526316
Epoch 3, Batch 390, Loss: 1.404546693349496, Accuracy: 0.25544871794871793
Epoch 3, Training Loss: 1.4048967599868774, Accuracy: 0.25506329113924053
Epoch 3, Accuracy: 0.2645569620253165
Epoch 4, Batch 10, Loss: 1.4418151736259461, Accuracy: 0.15
Epoch 4, Batch 20, Loss: 1.416103345155716, Accuracy: 0.18125
Epoch 4, Batch 30, Loss: 1.4116509477297465, Accuracy: 0.20833333333333334
Epoch 4, Batch 40, Loss: 1.4107915550470351, Accuracy: 0.215625
Epoch 4, Batch 50, Loss: 1.4099556994438172, Accuracy: 0.2325
Epoch 4, Batch 60, Loss: 1.4100184659163157, Accuracy: 0.23541666666666666
Epoch 4, Batch 70, Loss: 1.4068057349749974, Accuracy: 0.24107142857142858
Epoch 4, Batch 80, Loss: 1.4062302604317665, Accuracy: 0.2390625
Epoch 4, Batch 90, Loss: 1.4034376819928487, Accuracy: 0.24027777777777778
Epoch 4, Batch 100, Loss: 1.3998945260047913, Acc

Epoch 6, Batch 310, Loss: 1.406167455257908, Accuracy: 0.25120967741935485
Epoch 6, Batch 320, Loss: 1.4051085963845253, Accuracy: 0.253515625
Epoch 6, Batch 330, Loss: 1.4052227507938038, Accuracy: 0.25416666666666665
Epoch 6, Batch 340, Loss: 1.4056000285288868, Accuracy: 0.2540441176470588
Epoch 6, Batch 350, Loss: 1.4046321879114423, Accuracy: 0.2546428571428571
Epoch 6, Batch 360, Loss: 1.4034050034152137, Accuracy: 0.25555555555555554
Epoch 6, Batch 370, Loss: 1.4034149108706293, Accuracy: 0.25675675675675674
Epoch 6, Batch 380, Loss: 1.4034685235274464, Accuracy: 0.25723684210526315
Epoch 6, Batch 390, Loss: 1.4031871110965044, Accuracy: 0.25769230769230766
Epoch 6, Training Loss: 1.4029029994071285, Accuracy: 0.2572784810126582
Epoch 6, Accuracy: 0.2645569620253165
Epoch 7, Batch 10, Loss: 1.3482538104057311, Accuracy: 0.35
Epoch 7, Batch 20, Loss: 1.367399024963379, Accuracy: 0.33125
Epoch 7, Batch 30, Loss: 1.3781915823618571, Accuracy: 0.30416666666666664
Epoch 7, Batch 40, 

Epoch 9, Batch 250, Loss: 1.4078843760490418, Accuracy: 0.2395
Epoch 9, Batch 260, Loss: 1.4072089131061847, Accuracy: 0.24134615384615385
Epoch 9, Batch 270, Loss: 1.4063642753495111, Accuracy: 0.2412037037037037
Epoch 9, Batch 280, Loss: 1.4060553371906281, Accuracy: 0.24285714285714285
Epoch 9, Batch 290, Loss: 1.4065282743552636, Accuracy: 0.24267241379310345
Epoch 9, Batch 300, Loss: 1.4079483981927237, Accuracy: 0.24041666666666667
Epoch 9, Batch 310, Loss: 1.4071797332456035, Accuracy: 0.24233870967741936
Epoch 9, Batch 320, Loss: 1.4064304292201997, Accuracy: 0.244921875
Epoch 9, Batch 330, Loss: 1.4051096937873147, Accuracy: 0.24772727272727274
Epoch 9, Batch 340, Loss: 1.4061615915859447, Accuracy: 0.24889705882352942
Epoch 9, Batch 350, Loss: 1.4066531518527439, Accuracy: 0.2475
Epoch 9, Batch 360, Loss: 1.4057209644052717, Accuracy: 0.2482638888888889
Epoch 9, Batch 370, Loss: 1.40640526655558, Accuracy: 0.24763513513513513
Epoch 9, Batch 380, Loss: 1.4056017238842813, Accu

Epoch 12, Batch 180, Loss: 1.401626671685113, Accuracy: 0.25833333333333336
Epoch 12, Batch 190, Loss: 1.4011608945695977, Accuracy: 0.2611842105263158
Epoch 12, Batch 200, Loss: 1.4000096535682678, Accuracy: 0.263125
Epoch 12, Batch 210, Loss: 1.399136063598451, Accuracy: 0.2654761904761905
Epoch 12, Batch 220, Loss: 1.3981382749297402, Accuracy: 0.26704545454545453
Epoch 12, Batch 230, Loss: 1.3981114200923754, Accuracy: 0.26521739130434785
Epoch 12, Batch 240, Loss: 1.3985177829861641, Accuracy: 0.2625
Epoch 12, Batch 250, Loss: 1.398637411594391, Accuracy: 0.2595
Epoch 12, Batch 260, Loss: 1.3983550933691171, Accuracy: 0.2591346153846154
Epoch 12, Batch 270, Loss: 1.3987408227390714, Accuracy: 0.2574074074074074
Epoch 12, Batch 280, Loss: 1.3990499049425125, Accuracy: 0.25535714285714284
Epoch 12, Batch 290, Loss: 1.397808326112813, Accuracy: 0.25689655172413794
Epoch 12, Batch 300, Loss: 1.398215678135554, Accuracy: 0.25583333333333336
Epoch 12, Batch 310, Loss: 1.39820826092074, 

KeyboardInterrupt: 

---
## DistilBERT

In [49]:
# 필요한 라이브러리 추가
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

# 훈련 데이터셋과 테스트 데이터셋으로 분할
train_df, test_df = train_test_split(df_al, test_size=0.2, random_state=42)

# DistilBERT 토크나이저 로드
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=64):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['label']
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

# DataLoader 설정
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# DistilBERT 모델 로드
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=df_al['label'].nunique())

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
for epoch in range(3):  # 예시로 3 에폭만 진행
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_idx, batch in enumerate(train_dataloader):
        inputs = {
            "input_ids": batch["input_ids"],
            "attention_mask": batch["attention_mask"],
            "labels": batch["label"],
        }
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()

        # 정확도 계산
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == batch["label"]).sum().item()
        total_samples += len(batch["label"])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 한 batch가 끝날 때마다 출력
        if (batch_idx + 1) % 10 == 0:  # 예시로 10 배치마다 출력
            average_loss = total_loss / (batch_idx + 1)
            accuracy = correct_predictions / total_samples
            print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {average_loss}, Accuracy: {accuracy}")

    # 에폭이 끝날 때마다 출력
    average_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}, Training Loss: {average_loss}, Accuracy: {accuracy}")

    # 평가 루프
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in test_dataloader:
            inputs = {
                "input_ids": batch["input_ids"],
                "attention_mask": batch["attention_mask"],
                "labels": batch["label"],
            }
            outputs = model(**inputs)
            predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(batch["label"].cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Epoch {epoch + 1}, Accuracy: {accuracy}")


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

Epoch 1, Batch 10, Loss: 1.388614284992218, Accuracy: 0.275
Epoch 1, Batch 20, Loss: 1.3893455028533936, Accuracy: 0.28125
Epoch 1, Batch 30, Loss: 1.3917365789413452, Accuracy: 0.24166666666666667
Epoch 1, Batch 40, Loss: 1.3959510445594787, Accuracy: 0.23125
Epoch 1, Batch 50, Loss: 1.3981371712684632, Accuracy: 0.225
Epoch 1, Batch 60, Loss: 1.397163180510203, Accuracy: 0.23541666666666666
Epoch 1, Batch 70, Loss: 1.396534536566053, Accuracy: 0.24285714285714285
Epoch 1, Batch 80, Loss: 1.3980304718017578, Accuracy: 0.2359375
Epoch 1, Batch 90, Loss: 1.395051650206248, Accuracy: 0.24305555555555555
Epoch 1, Batch 100, Loss: 1.3926098370552062, Accuracy: 0.25125
Epoch 1, Batch 110, Loss: 1.390545094013214, Accuracy: 0.2534090909090909
Epoch 1, Batch 120, Loss: 1.3918190787235896, Accuracy: 0.25
Epoch 1, Batch 130, Loss: 1.3917047638159532, Accuracy: 0.25769230769230766
Epoch 1, Batch 140, Loss: 1.3910904509680613, Accuracy: 0.2598214285714286


KeyboardInterrupt: 

In [53]:
 !pip install wandb==0.16.0

     |████████████████████████████████| 2.1 MB 5.3 MB/s            
     |████████████████████████████████| 196 kB 73.1 MB/s            
     |████████████████████████████████| 254 kB 78.1 MB/s            
     |████████████████████████████████| 62 kB 2.4 MB/s             
     |████████████████████████████████| 143 kB 79.8 MB/s            
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
Note: you may need to restart the kernel to use updated packages.


In [168]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder

In [174]:
train_data_path ="~/aiffel/dktc/data/train.csv"
df = pd.read_csv(train_data_path)
df.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [71]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

In [72]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000, 
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[5527  210 5528 ... 5530   41 5531]
 [7965 7966    1 ...    1    1  396]
 [   1   96    1 ...    1  125    1]
 ...
 [ 485    3  113 ...   27 2015 5457]
 [   1    1  152 ...  350    1 3109]
 [   1    1    1 ...  156  114    1]] <keras_preprocessing.text.Tokenizer object at 0x7f49301a7b50>


In [73]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 내가
3 : 너
4 : 좀
5 : 다
6 : 왜
7 : 나
8 : 진짜
9 : 야
10 : 지금


In [74]:
X_train = tensor[:3500]
y_train = train_data['class'][:3500]
X_val = tensor[3500:3850]
y_val = train_data['class'][3500:3850]
X_test = tensor[3850:]
y_test = train_data['class'][3850:]

In [107]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertModel
from transformers import AdamW
from sklearn.metrics import accuracy_score



In [175]:

df = train_data
df.loc[ df['class']=='협박 대화', 'idx'] = 0
df.loc[ df['class']=='기타 괴롭힘 대화', 'idx'] = 3
df.loc[ df['class']=='갈취 대화', 'idx'] = 1
df.loc[ df['class']=='직장 내 괴롭힘 대화', 'idx'] = 2

df

,idx,class,conversation
0,0,3,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,3,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,1,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,0,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,1,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,0,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,2,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,0,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [176]:
df = df[['conversation','class']].rename(columns = {'class':'label','conversation':'text'})
df.head()

,text,label
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,3
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,3
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,1
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,0
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,0


In [113]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [165]:
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
config = transformers.DistilBertConfig(dropout=0.2, attention_dropout=0.2)
dbert_tf = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config, trainable=False)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [177]:


import tensorflow as tf
X= df['text']
y = df['label']
# DistilBertTokenizer로 모든 문장 토큰화
X_tf = [tokenizer(text, padding='max_length', max_length = 225, truncation=True)['input_ids'] for text in X]
X_tf = np.array(X_tf, dtype='int32')
# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2, random_state=42, stratify=y)
print('Shape of training data: ',X_train.shape)
print('Shape of test data: ',X_test.shape)
# Get BERT layer
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
config = transformers.DistilBertConfig(dropout=0.2, attention_dropout=0.2)
dbert_tf = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config, trainable=False)

Shape of training data:  (3160, 225)
Shape of test data:  (790, 225)


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [84]:
wandb.login(key = '0d59d7a17cbfbc47091fd5af0f49a1908e832c2e')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [178]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [187]:
import tensorflow as tf
from tensorflow import keras
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras import models, layers, metrics

def train():
    default_config = {
    "vocab": 10000,
    "units_1": 64,
    "units_2": 32,
    "units_3" : 32,
    "class_num": 4,
    "learning_rate": 0.001,
    "optimizer": "adam",
    "loss": "sparse_categorical_crossentropy",
    "metrics": ["accuracy"],
    "epoch": 10,
    "batch_size": 16,
    "dropout": 0.5
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model
    # Get BERT layer
    # 입력 정의
      # Model
    #input_ids_in = layers.Input(shape=(max_length,), dtype='int32')
    #attention_mask_in = layers.Input(shape=(max_length,), dtype='int32')

    input_ids_in = keras.layers.Input(shape=(225,), name='input_token', dtype='int32')

    x = dbert_tf(input_ids=input_ids_in)[0][:,0,:]
    x = keras.layers.Dropout(config['dropout'], name='dropout')(x)
    x = keras.layers.Dense(config['units_1'], activation='relu')(x)   
    #x = keras.layers.LSTM(units=config['units_2'])(x)
    x = keras.layers.Dense(config['class_num'], activation='softmax', name='classification')(x)

    model = models.Model(inputs=input_ids_in, outputs = x, name='ClassificationModelTF')


    # Concatenate DistilBERT output and LSTM output
    #combined_model = layers.concatenate([distilbert_output, lstm_model.output])
    #output_layer = layers.Dense(config['class_num'], activation='softmax')(combined_model)

    # Final Model
    #model = keras.Model(inputs=[input_ids_in, attention_mask_in], outputs=output_layer)

    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=config['learning_rate'])
    model.compile(optimizer=optimizer, loss=config['loss'], metrics=config['metrics'])

    # Train the model
    model.fit(
        x=X_train,
        y=y_train,
        epochs=config['epoch'],
        batch_size=config['batch_size'],
        validation_split=0.2,
        callbacks=[WandbCallback()]
    )

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

    # Log test metrics to WandB
    wandb.log({"Test Accuracy Rate": round(test_accuracy * 100, 2),
               "Test Error Rate": round((1 - test_accuracy) * 100, 2)})

In [188]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'yunjiyeong0106',
                       project = 'aiffel')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: o4ls6xsh
Sweep URL: https://wandb.ai/yunjiyeong0106/aiffel/sweeps/o4ls6xsh


wandb: Agent Starting Run: w4fg8wp7 with config:
wandb: 	epoch: 10
wandb: 	learning_rate: 0.05146482669207599


Epoch 1/10
  6/158 [>.............................] - ETA: 19s - loss: 11.7039 - accuracy: 0.2604WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0074s vs `on_train_batch_end` time: 0.1296s). Check your callbacks.


158/158 [==============================] - 31s 176ms/step - loss: 1.8082 - accuracy: 0.2532 - val_loss: 1.3927 - val_accuracy: 0.2563


wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


...

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /aiffel/aiffel/Aiffel_online_Quest/DL/wandb/run-20240110_153339-0xjus0z7/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/Aiffel_online_Quest/DL/wandb/run-20240110_153339-0xjus0z7/files/model-best/assets
/opt/conda/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
wandb: Adding directory to artifact (/aiffel/aiffel/Aiffel_online_Quest/DL/wandb/run-20240110_153339-0xjus0z7/files/model-best)... Done. 3.1s


Epoch 2/6
158/158 [==============================] - 26s 163ms/step - loss: 1.3883 - accuracy: 0.2599 - val_loss: 1.3925 - val_accuracy: 0.2389
Epoch 3/6
158/158 [==============================] - 26s 167ms/step - loss: 1.3896 - accuracy: 0.2627 - val_loss: 1.3955 - val_accuracy: 0.2563
Epoch 4/6
158/158 [==============================] - 26s 162ms/step - loss: 1.3895 - accuracy: 0.2611 - val_loss: 1.4005 - val_accuracy: 0.2563
Epoch 5/6
158/158 [==============================] - 26s 163ms/step - loss: 1.3865 - accuracy: 0.2686 - val_loss: 1.3973 - val_accuracy: 0.2389
Epoch 6/6
158/158 [==============================] - 26s 165ms/step - loss: 1.3868 - accuracy: 0.2714 - val_loss: 1.3912 - val_accuracy: 0.2389
25/25 - 6s - loss: 1.3901 - accuracy: 0.2481


wandb: Ctrl + C detected. Stopping sweep.
